In [36]:
import java.nio.file.Files
import java.nio.file.Paths

fun String.createMatrix() : Array<IntArray> {

    return this.split('\n')
        .filter { it.isNotEmpty() }
        .map { line ->
            line.map { char ->
                when (char) {
                    '@' -> 1
                    '.' -> 0
                    else -> throw IllegalArgumentException("Invalid character in input: $char")
                }
            }.toIntArray()
        }.toTypedArray()
}

fun Array<IntArray>.print() {
    for (row in this) {
        println(row.joinToString(" "))
    }
}

fun Array<IntArray>.convolute(
    kernel: Array<IntArray>
): Array<IntArray> {
    val rows = this.size
    val cols = this[0].size
    val output = Array(rows) { IntArray(cols) }

    val kernelSize = kernel.size
    require(kernelSize == kernel[0].size) { "Kernel must be square" }
    require(kernelSize % 2 == 1) { "Kernel size must be odd" }

    val radius = kernelSize / 2

    for (i in 0 until rows) {
        for (j in 0 until cols) {
            var sum = 0
            for (ki in 0 until kernelSize) {
                for (kj in 0 until kernelSize) {
                    val ni = i + ki - radius
                    val nj = j + kj - radius
                    if (ni in 0 until rows && nj in 0 until cols) {
                        sum += this[ni][nj] * kernel[ki][kj]
                    }
                }
            }
            output[i][j] = sum
        }
    }
    return output
}

operator fun Array<IntArray>.times(b: Array<IntArray>): Array<IntArray> {
    val rows = this.size
    val cols = this[0].size
    val output = Array(rows) { IntArray(cols) }

    require (rows == b.size)
    require (cols == b[0].size)

    for (i in 0 until rows) {
        for (j in 0 until cols) {
            output[i][j] = this[i][j] * b[i][j]
        }
    }
    return output
}

fun Array<IntArray>.applyOp(op: (Int) -> Int) : Array<IntArray> {
    val rows = this.size
    val cols = this[0].size
    val output = Array(rows) { IntArray(cols) }
    for (i in 0 until rows) {
        for (j in 0 until cols) {
            output[i][j] = op(this[i][j])
        }
    }
    return output
}

fun Array<IntArray>.countNonZero() : Int {
    val rows = this.size
    val cols = this[0].size
    var count = 0
    for (i in 0 until rows) {
        for (j in 0 until cols) {
            if (this[i][j] > 0) count++
        }
    }
    return count
}

In [16]:

val use_input = false
val debug = true

val m = if (use_input) {
    Files.readString(Paths.get("input.txt"))
} else {
    """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
"""
}.createMatrix()


m.print()

val kernel = """@@@
@.@
@@@""".createMatrix()

kernel.print()

0 0 1 1 0 1 1 1 1 0
1 1 1 0 1 0 1 0 1 1
1 1 1 1 1 0 1 0 1 1
1 0 1 1 1 1 0 0 1 0
1 1 0 1 1 1 1 0 1 1
0 1 1 1 1 1 1 1 0 1
0 1 0 1 0 1 0 1 1 1
1 0 1 1 1 0 1 1 1 1
0 1 1 1 1 1 1 1 1 0
1 0 1 0 1 1 1 0 1 0
1 1 1
1 0 1
1 1 1


In [38]:
val iteration = (m.convolute(kernel) * m).applyOp { if (it>3) 1 else 0}
m.countOnes() - iteration.countOnes()

13

In [44]:
var current = m
var currentCount = m.countOnes()
do {
    currentCount = current.countOnes()
    current = (current.convolute(kernel) * current).applyOp { if (it>3) 1 else 0}
} while (currentCount > current.countOnes())
m.countOnes()-current.countOnes()

43